# Twitter Timeline / User Profile Download

Download twitter timelines and save to disk locally, for model training purposes.  
Download twitter model user profiles, for quick save (i.e. user profile urls)

### Imports

In [ ]:
import os
os.chdir('..')

In [ ]:
import datetime
import pickle
import itertools
import json

In [ ]:
import assembly
from assembly import config as asmbl_config
from assembly import db as asmbl_db
from assembly import models as asmbl_models

In [ ]:
from lib.tweemio import twitter
from lib.tweemio import similarity

### Initializations

In [ ]:
#  Assembly initialization - will use configs defined for env in question
os.environ['ASSEMBLY_ENV'] = 'Development'
app = assembly.Assembly.init(__name__, {'default': []})

In [ ]:
twapi = twitter.TwitterApi(asmbl_config['TWITTER_API_CREDS'])

### User Inputs

In [ ]:
userprofile_save_location = '/Users/liangjh/workspace/tweemio-api/resources/model_user_profiles.json'
timelines_save_location = '/Users/liangjh/workspace/tweemio-api/data/mdl-timelines/'

In [ ]:
#  Full list of twitter handles, across multiple categories
#  Download full timelines (or as much as the API will give us)
twitter_handles = list(itertools.chain(*[spec['screen_names'] 
                                         for category,spec in app.config['SIMILARITY_COMPARISONS'].items()]))

### Timeline Download, Save to Disk

In [ ]:
twitter_handles.index('johnlegendbillmaher')

In [ ]:
#  Serialize to JSON, save w/ timestamp
timeline_map = {}
for screen_name in twitter_handles:
    print(f'Downloading timeline for screen name: {screen_name}')
    user_timeline = twapi.timeline_raw(screen_name)
    timeline_map[screen_name] = user_timeline

In [ ]:
for screen_name, tline in timeline_map.items():
    print(f'writing {screen_name} to file...')
    tline_js = [tli._json for tli in tline]
    with open(f"{timelines_save_location}/{datetime.date.today().strftime('%Y%m%d')}__{screen_name}.json", "w") as outfile:
        json.dump(tline_js, outfile)

### User Profile Download, Persist to JSON

Users that are used to calibrate model and provide similarity scores against.   
Save some of their profile information to a JSON to allow front-end to render 

In [ ]:
# ** If Needed **
#  Load timelines (for readability analysis)
# timeline_map = {}
# for screen_name in twitter_handles:
#     with open(f"{timelines_save_location}/{datetime.date.today().strftime('%Y%m%d')__{screen_name}}.json") as f:
#         tline_data = json.load(f);
#     timeline_map[screen_name] = tline_data

In [ ]:
user_profile_map = {}
for screen_name in twitter_handles:
    print('getting user: ' + screen_name)
    usr = twapi.user(screen_name=screen_name)   

    #  Calc readability score, save
    tline = timeline_map[screen_name]
    timeline_text  = list(reversed([tli._json['full_text'] for tli in tline]))
    timeline_text =  twitter.TwitterApi.timeline_filter(timeline_text, condense_factor=asmbl_config['TWEET_CONDENSE_FACTOR'])
    readability = similarity.mdl_readability_scores(timeline_text)    

    user_profile_map[screen_name] = {
        'user': {
            'screen_name': screen_name,
            'name': usr.name,
            'desc': usr.description,
            'profile_img': usr.profile_image_url_https
        },
        'readability': readability
    }

In [ ]:
with open(userprofile_save_location, 'w') as outfile:
    json.dump(user_profile_map, outfile)